In [13]:
import pyTigerGraph as tg
from pyTigerGraph.pyTigerGraph import TigerGraphException
import pandas as pd

# Create & Test Connection

In [47]:
secret = '9jgvbu34uq8sgjg7bn4fe5mrhiokvfcf'
cred = {
    "host": "https://gradient-test.i.tgcloud.io",
    "graphname": "gradient",
    "username": "tigergraph",
    "password": "Gradient123"
}


token, token_life_sec, token_expir_date = tg.TigerGraphConnection(**cred).getToken(secret="9jgvbu34uq8sgjg7bn4fe5mrhiokvfcf")

cred["apiToken"] = token
conn = tg.TigerGraphConnection(**cred)
print(conn.gsql('ls')) # test connection

---- Graph gradient
Vertex Types:
- VERTEX Drug(PRIMARY_ID drug_id STRING, common_name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Disease(PRIMARY_ID disease_id STRING, common_name STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Name(PRIMARY_ID name STRING, isCommon BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- UNDIRECTED EDGE interacts_with(FROM Drug, TO Disease)
- UNDIRECTED EDGE has_name(FROM Drug, TO Name|FROM Disease, TO Name)

Graphs:
- Graph gradient(Drug:v, Disease:v, Name:v, interacts_with:e, has_name:e)
Jobs:
- CREATE SCHEMA_CHANGE JOB initGraph FOR GRAPH gradient {
ADD VERTEX Drug(PRIMARY_ID drug_id STRING, common_name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD VERTEX Disease(PRIMARY_ID disease_id STRING, common_name STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE=

# Create Graph

In [21]:
graph_not_exists_message = "Graph name gradient cannot be found. Please provide a valid graph name via 'graph=xxx' parameter string."

try:
    conn.getSchema() # Check if graph exists
    print(conn.gsql('DROP GRAPH gradient'))
except TigerGraphException as e:
    print(e.message)

Graph name gradient cannot be found. Please provide a valid graph name via 'graph=xxx' parameter string.


In [23]:
create_graph_query_fname = "../core/graph/create_graph_schema.gsql"

with open(create_graph_query_fname, "r") as f:
    create_graph_query = f.read()

print(conn.gsql(create_graph_query))

The graph gradient is created.
Using graph 'gradient'
Successfully created schema change jobs: [initGraph].

Current graph version 0
Trying to add vertex Drug.
Trying to add vertex Disease.
Trying to add vertex Name.
Trying to add edge interacts_with.
Trying to add edge has_name.
Kick off job initGraph

Graph gradient update to new version 1
The job initGraph completes in 11.427 seconds!


# Load Graph

In [55]:
data_dir = '../core/dataset/output'

In [67]:
drug_df = pd.read_csv(f'{data_dir}/drug.tsv', sep='\t')
drug_df.fillna('', inplace=True)

conn.upsertVertexDataFrame(
    drug_df,
    'Drug',
    v_id = 'DrugBank ID',
    attributes = {
        'common_name': 'Common name'
    }
)

14594

In [69]:
disease_df = pd.read_csv(f'{data_dir}/disease.tsv', sep='\t')
disease_df.fillna('', inplace=True)

conn.upsertVertexDataFrame(
    disease_df,
    'Disease',
    v_id = 'Disease Mesh ID',
    attributes = {
        'common_name': 'Name',
        'description': 'Description'
    }
)

11332

In [71]:
drug_disease_df = pd.read_csv(f'{data_dir}/disease-drug.tsv', sep='\t')

conn.upsertEdgeDataFrame(
    drug_disease_df,
    'Drug',
    'interacts_with',
    'Disease',
    from_id = 'Chemical',
    to_id = 'Disease Mesh ID',
    attributes = {}
)

466657

In [77]:
drug_to_name_df = pd.read_csv(f'{data_dir}/drug-name.tsv', sep='\t')
drug_names_df = drug_to_name_df[['Name', 'isCommon']].drop_duplicates()

print(conn.upsertVertexDataFrame(
    drug_name_df,
    'Name',
    attributes={
        'name': 'Name',
        'isCommon': 'isCommon'
    }
))

print(conn.upsertEdgeDataFrame(
    drug_to_name_df,
    'Drug',
    'has_name',
    'Name',
    attributes={}
))

42016
42016


In [ ]:
disease_to_name_df = pd.read_csv(f'{data_dir}/disease-name.tsv', sep='\t')
disease_names_df = disease_to_name_df['Name', 'isCommon'].drop_duplicates()

print(conn.upsertVertexDataFrame(
    disease_name_df,
    'Name',
    attributes={
        'name': 'Name',
        'isCommon': 'isCommon'
    }
))

print(conn.upsertEdgeDataFrame(
    disease_to_name_df,
    'Disease',
    'has_name',
    'Name',
    attributes={}
))

In [ ]:
# drug ratings
drug_ratings_df = pd.read_csv(f'{data_dir}/', sep='\t')

In [ ]:
# drug provider namessu